# DataLab Python for Data Analysis Walkthrough. 


### A few notes before we get started.

This project is on github. To download enter the following on the commandline.
```
git clone https://github.com/asfox/DataLab_PythonWorkshop_Olympics.git

```

I used [cookiecutter](https://cookiecutter.readthedocs.io/en/1.7.0/index.html) to organize this project. I'd recommend you do the same with your own projects. For something like this, it's probably overkill, but it's a good habit to start. To install cookiecutter, run the following on the commandline. 

``` 
pip install cookiecutter

cookiecutter https://github.com/drivendata/cookiecutter-data-science
# then follow the prompts.

```

# Intro to Jupyter Notebook
### this is _Markdown_

# Getting started

First thing to do is import some packages. 

Packages are just objects that you get from other files. Objects can have functions and hold data. Often when the object gets super complicated, people package it together into a nice, clean oject with some accompanying documentation. 

Not having all possible packages loaded by default allows python to load and run faster, only taking the time for things you need. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re as re

**pandas** will allow you to use DataFrames in python. 

**matplotlib** is the workhorse of plotting in python. 

**seaborn** is effectively a wrapper for matplotlib that makes science plotting easier. 

**numpy** will do most mathamatical operations (and matrix operations) in python (QUICKLY!)

**re** will allow you to use regular expressions in python. 

# Load our first dataset. 

This is data on olympians from a csv I found online. It's in the github. 

In [ ]:
df_olympics = pd.read_csv('../data/raw/athlete_events.csv')

_Pro tip:_ make sure you get the path right!

In [ ]:
df_olympics.head()

### Subset and organize your dataset. 

In [ ]:
df_olympics = df_olympics[df_olympics['Season']=='Summer']

In [ ]:
df_winners = df_olympics.groupby(['Team', 'Medal', 'Year']).count()['ID']

In [ ]:
df_winners.head()

In [ ]:
df_winners.unstack('Medal')
df_winners = df_winners.unstack('Medal').fillna(0).reset_index()


_Pro tip:_ pandas can stack, unstack, melt, and piviot. If those mean something to you, great. If not, these are good things to google, when you are stuck trying to re-form a DataFrame. 

In [ ]:
df_winners.head()

### Add variables. 

In [ ]:
df_winners['Total Medals'] = df_winners[['Bronze', 'Gold', 'Silver']].sum(axis=1)


_Pro tip:_ It's good practice to keep all your data manipulations in the same place. This way, you don't run into trouble overwriting/changing data when you run cells in the wrong order.

### Plot your data. 

In [ ]:
f, ax = plt.subplots(1,1,figsize=(16,4))
ax = sns.barplot(x='Team', y='Gold', data=df_winners.loc[df_winners['Year']==1992])
ax.set_xticklabels(ax.get_xticklabels(),rotation=90);


In [ ]:
f, ax = plt.subplots(1,1,figsize=(16,4))
ax = sns.swarmplot(x='Team', y='Gold', data=df_winners.loc[df_winners['Year']>=1981])
ax.set_xticklabels(ax.get_xticklabels(),rotation=90);



In [ ]:
sns.scatterplot('Gold', 'Bronze', data=df_winners)

In [ ]:
f, ax = plt.subplots(1,1,figsize=(16,4))
ax = sns.barplot(x='Team', y='Total Medals', data=df_winners.loc[df_winners['Year']==1992])
ax.set_xticklabels(ax.get_xticklabels(),rotation=90);




## Challenge:

Do the same countries win the winter olympics? 

***

# Are winners just big countries? 

Load, clean, and merge our next dataset. 

### Load population data.

In [ ]:
df_pop = pd.read_csv('../data/raw/population-figures-by-country-csv.csv')
df_pop.head()

### Clean population data. 

As a first step, it's annoying to have variables named "Year_1961". Let's just fix that with a regular expression. 

Here I'm passing a function to the rename function -- I know. Fancy!

This is the kind of thing you can usually just find on stackoverflow (like I did!).

In [ ]:
df_pop = df_pop.rename(columns=lambda x: re.sub('^Year_','',x))
df_pop.head()

In [ ]:
df_pop = df_pop.melt(id_vars=['Country', 'Country_Code'])
df_pop.columns = ['Country', 'Country_Code', 'Year', 'Population']

_Pro tip:_ **One of the most common problems people run into involve trying to do this without thinking about it.**

Here's another place where I used a DataFrame function to reshape my data. It really helps to have in mind what you want your new DataFrame to look like. I usually start out with a drawing on paper or a picture in my mind. Once you have that, you can run the command you want (or google for it)
    
    

In [ ]:
# df_winners.head()
df_pop.tail()


### Merge olympic data with population data. 

In [ ]:
df_pop_winner = df_winners.merge(df_pop, left_on=['Team','Year'], 
                                 right_on=['Country', 'Year'], how='inner' )

### Error messeges!

Make sure to read your error messeges from bottom to top. 

The last line will tell you the error. The lines above it will give you some context about the code that ran the errant code. The lines above that will give you some context about the lines that ran the code that ran the errant code. And so on. 

This particular error is a version of the notorious _TypeError_ that I mentioned... In this case it get's called a _ValueError_ but the problem is that I'm trying to merge files of different types. 

In [ ]:
print(df_pop['Year'].dtype, print(df_winners['Year'].dtype ) )
# df_pop['Year'].values

### Fix the error by doing a better job at cleaning data

If this weren't a tutorial, I'd add this code above, before I tried to merge. 

In [ ]:
df_pop['Year'] = pd.to_numeric(df_pop['Year'])
print(df_pop['Year'].dtype, print(df_winners['Year'].dtype ) )

### Now merge data. 

In [ ]:
df_pop_winner = df_winners.merge(df_pop, left_on=['Team','Year'], 
                                 right_on=['Country', 'Year'], how='inner' )

In [ ]:
df_pop_winner.head()

### Plot merged data

Now we can plot medals against population across contries. 

In [ ]:
sns.scatterplot('Bronze', 'Population', data=df_pop_winner)

Maybe the distribution is messing us up? Let's log-scale and try again... 

In [ ]:
df_pop_winner['log10(Population)'] = np.log10(df_pop_winner['Population'])

In [ ]:
sns.scatterplot('Bronze', 'log10(Population)', data=df_pop_winner)

## Challenge:
Can you re-organize your version of the notebook so that we avoid erorrs and edit our data all in one place? 

***

# Are rich countries winning?

I downloaded GDP data from online. This data is also in the github. 

https://data.worldbank.org/indicator/NY.GDP.MKTP.CD

### load the data. 

In [ ]:
df_gdp_raw = pd.read_csv('../data/raw/API_NY/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_820841.csv', 
                  skiprows=3)

In [ ]:
df_gdp_raw.head()

### Clean the GDP data

In [ ]:
print(df_gdp_raw.columns)
df_gdp_raw.drop( columns='Unnamed: 64', inplace=True )

This is closer to how a good notebook _**should**_ look. 

In [ ]:
df_gdp = df_gdp_raw.melt(id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'])

# select only GDP in current USD
df_gdp = df_gdp[ df_gdp['Indicator Name'] == 'GDP (current US$)' ]

# select columns, and fix names
df_gdp = df_gdp[['Country Name', 'Country Code', 'Indicator Name', 'variable', 'value']]
df_gdp.columns = ['Country Name', 'Country Code', 'Indicator Name', 'Year', 'GDP']

# convert year to numeric
df_gdp['Year'] = pd.to_numeric( df_gdp['Year'] )

# convert GDP to numeric
df_gdp['GDP (in USD)'] = pd.to_numeric( df_gdp['GDP'] )


In [ ]:
df_gdp.head()

### Merge olympic medal data and GDP data. 

In [ ]:
df_gdp_winner = df_winners.merge(df_gdp, left_on=['Team','Year'], 
                                 right_on=['Country Name', 'Year'], how='inner' )

In [ ]:
df_gdp_winner.head()

### Plot olympic medals and GDP.

In [ ]:
sns.scatterplot('Bronze', 'GDP', data=df_gdp_winner)

### create new variables

_If_ we were doing this not as a tutorial, this should be moved up, before we merge and plot. For a workshop or tutorial, I think it's clearer here. 

In [ ]:
df_gdp_winner['log10(GDP)'] = np.log10(df_gdp_winner['GDP'])

### Plot log-scaled medals. 

In [ ]:
sns.scatterplot('Total Medals', 'log10(GDP)', data=df_gdp_winner)

In [ ]:
# create three subplots using matplotlib
f, ax_list = plt.subplots(1,3, figsize=(15,5))

# plot each medal on a different subplot
sns.scatterplot('Bronze', 'log10(GDP)', data=df_gdp_winner, ax=ax_list[0], color=sns.xkcd_rgb['bronze'])
sns.scatterplot('Silver', 'log10(GDP)', data=df_gdp_winner, ax=ax_list[1], color=sns.xkcd_rgb['silver'])
sns.scatterplot('Gold', 'log10(GDP)', data=df_gdp_winner, ax=ax_list[2], color=sns.xkcd_rgb['gold'])



In [ ]:
f, ax = plt.subplots(1,1,figsize=(5,5))

# if we pass the same axis, to each plot, we can make them overlap. 
bronze = sns.scatterplot('Bronze', 'log10(GDP)', data=df_gdp_winner, ax=ax, alpha=.3, color=sns.xkcd_rgb['brown'])
silver = sns.scatterplot('Silver', 'log10(GDP)', data=df_gdp_winner, ax=ax, alpha=.3, color=sns.xkcd_rgb['silver'])
gold = sns.scatterplot('Gold', 'log10(GDP)', data=df_gdp_winner, ax=ax, alpha=.3, color=sns.xkcd_rgb['gold'])

# set our axis x-label
ax.set_xlabel('Medals')

# Add a legend. Because we did this across different plots, 
# we need to create a custom legend. Custom legends can be annoying. 
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', label='Bronze',
                          markerfacecolor=sns.xkcd_rgb['brown'], markersize=10),
                   Line2D([0], [0], marker='o', color='w', label='Silver',
                          markerfacecolor=sns.xkcd_rgb['silver'], markersize=10),
                  Line2D([0], [0], marker='o', color='w', label='Gold',
                          markerfacecolor=sns.xkcd_rgb['gold'], markersize=10),]

# In an ideal world, we would only need this line. 
#   That would have been the case if we were clever and 
#   found a way to make this using one plot command.
ax.legend(handles=legend_elements, loc='lower right')

# save the figure into our reports/figures!
plt.savefig( '../reports/figures/Log10GDP_vs_OlympicMedals.png', bbox_inches = "tight")

# Save our processsed data. 

Merge all the data that we've worked with into one dataframe. 

In [ ]:
df_pop_gdp_winner = df_pop_winner.merge(df_gdp, left_on=['Team','Year'], 
                                 right_on=['Country Name', 'Year'], how='inner' )
df_pop_gdp_winner.head()

Save the merged dataframe as as .csv file in the processed data foler

In [ ]:
df_pop_gdp_winner.to_csv('../data/processed/merged_winner_pop_gdp.csv', index=False)

_Pro tip:_ if you don't need your index, don't save it!